TO DO:
- Make generic functions for tasks for modularity
- CRON jobs for all timeframes
- One function to update all csvs
- Plug n play for indicators
- Add limit to queries
- Include Active/Inactive column in CSV for active/inactive coins
- Update_CSV_to_Latest should contain active coins from Exchanges and From CSV. Check if a (coin,exchange) tuple is active(Check if it is present on exchange using ccxt library) . If it is active   get latest data for it if already present in CSV,if not in CSV get all data. If coin is not active on exchange , we will put a Active/Inactive status in CSV accordingly. All functions will have   to be modified to run code only for active coin-exchange combinations.
- Date Format will be '%d-%m-%Y %H:%M:%S' . This is giving me a lot of problems especially while reading data. When I don't put :%S it tells me dataframe has second and sometimes when :%S is         there, it tells me no second value in dataframe.
- Analytics Value Accuracy. Some parameter in Jupyter.
- Have to fetch Coins based on Parameters. Example - Fetch all active coins-exchange combinations where RSI>0 and RSI<=30. Fetch all active coins-exchange combinations where closing price is         between LOWERBAND and MIDDLEBAND. Get me intersection(common coins) of these 2 list. Now the coin from the intersection list which probably has the lowest volume can increase in price faster       then the others(Little increase in Volume will result in Big increase in Price)
- For each active coin-exchange combination I want to check the change in Value of different Technical Indicators of 2 consecutive periods in time. Example - I want to know if for a particular       coin RSI=a on period x and RSI>a on period x+1. I want to know whenever MACD and MACD_SIGNAL cross each other(On period x MACD=a and MACD_SIGNAL=b where a<=b and on period x+1 MACD=a and           MACD_SIGNAL=b where a>b. MACD_HISTOGRAM same like RSI want to know when it is 'a' on period x and 'a++' on period x+1).
- Convert 1D timeframe to 3D/1Week/etc. Convert 1H timeframe to 4H/6H/etc.
- Batch Processing of downloading new coins data

In [108]:
# Imports
import os
import time
import talib
import logging
import requests
import datetime
import importlib
import dateutil.parser
import ccxt
import numpy as np
import pandas as pd
from importlib import reload
from collections import defaultdict
import sys
import inspect
import pyti
from apscheduler.schedulers.background import BackgroundScheduler

In [109]:
importlib.reload(logging)
LOGGING_FORMAT = '%(asctime)s - %(levelname)s - %(message)s'
logging.basicConfig(format=LOGGING_FORMAT, level=logging.
                    INFO)


In [110]:
cryptocompare_wrapper = 'D:\\crypto_analysis-master-master\\crypto_analysis-master-master\\notebooks\\cryptocompare_wrapper.py'
cryptocompare_wrapper

'D:\\crypto_analysis-master-master\\crypto_analysis-master-master\\notebooks\\cryptocompare_wrapper.py'

In [111]:
import cryptocompare_wrapper as ccw
reload(ccw)

<module 'cryptocompare_wrapper' from 'D:\\crypto_analysis-master-master\\crypto_analysis-master-master\\notebooks\\cryptocompare_wrapper.py'>

In [112]:
# FIELDS
PRICE = 'PRICE'
HIGH = 'HIGH24HOUR'
LOW = 'LOW24HOUR'
VOLUME = 'VOLUME24HOUR'
CHANGE = 'CHANGE24HOUR'
CHANGE_PERCENT = 'CHANGEPCT24HOUR'
MARKETCAP = 'MKTCAP'
NPERIODS = 100
TIMEFRAME = 'Day'
datetimeStringformat_to_csv = "%d-%m-%Y %H:%M"
data_directory = 'D:\\crypto_analysis-master-master\\crypto_analysis-master-master\\data\\'

In [113]:
# Defaults
CURR = 'BTC'
EXCHANGE = 'CCCAGG'
COIN = 'ETH'
COIN_LIST = ['BTC', 'ETH', 'XRP']
EXCHANGES = ['Bittrex','Binance','Kucoin','HuobiPro','Cryptopia','IDEX']
EXCHANGES = ['Binance']
bittrex_exchange = ccxt.bittrex()
binance_exchange = ccxt.binance()
kucoin_exchange = ccxt.kucoin()
huobiPro_exchange = ccxt.huobipro()
cryptopia_exchange = ccxt.cryptopia()
bitmex_exchange = ccxt.bitmex()
#print(bittrex_exchange.fetchCurrencies())
list_of_exchanges = [bittrex_exchange,binance_exchange,kucoin_exchange,huobiPro_exchange,
                     cryptopia_exchange]
#list_of_exchanges = [bitmex_exchange]

In [114]:
# For every exchange, fetch it's markets. Then depending on the JSON returned, prepare a list of coins for which historical data has to be downloaded.
def setupExchanges(list_of_exchanges):
    done = False
    i=0
    #df_markets = pd.DataFrame(markets)
    #bittrex_market = bittrex_exchange.fetchMarkets()
    #binance_market = binance_exchange.fetchMarkets()
    #kucoin_market = kucoin_exchange.fetchMarkets()
    #list_of_markets = [#bittrex_market,
                       #binance_market
     #                  kucoin_market #For kucoin the fetchMarkets function returns different dictionary keys
      #                  ]

    var_quote = ""
    coin_exchange_combination = {}
    for exchange in list_of_exchanges:
        coins_list = set()
        #if exchange.name == 'Cryptopia' or exchange.name == 'Bittrex' or exchange.name == 'Kucoin' or exchange.name == 'Huobi Pro':
            #continue #exchange.name == 'Binance' or 
        markets = exchange.fetchMarkets()
        for row in markets:
            if exchange.name == 'Huobi Pro' or exchange.name == 'Cryptopia':
                if row['base'] not in coins_list:
                        coins_list.add(row['base'])
                #continue
            if  'active' in row and row['active'] == True :
                #print(exchange.name,row)
                #sys.exit("Te")
                if exchange.name == 'Bittrex' or exchange.name == 'Binance'  :
                    var_quote = "quoteId"
                elif   exchange.name == 'Kucoin' or exchange.name == 'Huobi Pro':
                    var_quote = "quote"
                #print(var_quote)
                if var_quote in row and row[var_quote] == 'BTC':
                    if row['base'] not in coins_list:
                        coins_list.add(row['base'])
        coin_exchange_combination[exchange.name] = coins_list
    return coin_exchange_combination

In [115]:
def convert_timestamp(timestamp):
    """ Convert timestamp into readable datetime """
    try:
        return datetime.datetime.fromtimestamp(int(timestamp)).strftime('%d-%m-%Y %H:%M')
        #return timestamp
    except Exception as e:
        logging.debug(e)
        return None

In [116]:
def update_and_delete_coin_exchange_combination(csv_filename_read,csv_filename_write,timeframe,coin_exchange_combination):
    #For every exchange, download the new coins and delete coins which are delisted from that exchange
    csv_all_coins_full = csv_filename_read
    csv_all_coins_full_new = csv_filename_write
    not_updated = defaultdict(list)
    existing_coin_exchange = []
    # If the csv already exists, find out which coins and exchanges have already been added
    if os.path.isfile(csv_all_coins_full):
        df_csv_all_coins_full = pd.read_csv(csv_all_coins_full,index_col=['exchange','coin'])
        # existing_coin_exchange is a list of tuples (coin, exchange)
        existing_coin_exchange = np.unique(df_csv_all_coins_full.index.values)
        coin_exchange_combination_in_excel = {}
        for a, b in existing_coin_exchange:
            coin_exchange_combination_in_excel.setdefault(a, []).append(b)
    number_of_coins = 0
    coin_exchange_combination_to_delete = {}
    for exchange in coin_exchange_combination :
        if exchange not in coin_exchange_combination_in_excel:
            continue
        coins_list_from_exchange = coin_exchange_combination[exchange]
        coins_list_from_excel = coin_exchange_combination_in_excel[exchange]
        coins_to_download = list(set(coins_list_from_exchange) - set(coins_list_from_excel))
        coins_to_delete = list(set(coins_list_from_excel) - set(coins_list_from_exchange))
        coin_exchange_combination_to_delete[exchange] = coins_to_delete
        print(exchange,coins_to_download)
        for symbol in coins_to_download:
         #For every symbol-exchange combination, if it is present in CSV,don't download historical Data for it.
            try:
                # Can't fetch the same symbol in same symbol rate
                print(symbol,exchange,"In Try Block")
                func = function_period_mapping[timeframe]
                to_curr = 'BTC'
                if symbol == "BTC":
                    to_curr = "USD"
                if exchange == 'IDEX':
                    to_curr = 'ETH'
                if symbol is not to_curr:
                    df_coin_all = func(
                        coin=symbol,
                        to_curr=to_curr,
                        timestamp=time.time(),
                        exchange=exchange
                    )
    
                if df_coin_all.empty:
                    not_updated[exchange].append(symbol)
                    #print(symbol,exchange)
                else:
                    df_coin_all['exchange'] = exchange
                    df_coin_all['coin'] = symbol
                    print("Coin Inserted")
                    df_coin_all = df_coin_all.reset_index()
                    df_coin_all['time'] = df_coin_all.unix_timestamp.apply(convert_timestamp)
                    print("Time column created")
                    df_coin_all = df_coin_all.set_index(['coin','exchange', 'unix_timestamp'])
                    print("Index Set Again")
                    # If csv does not exist, write, else append
                    if not os.path.isfile(csv_all_coins_full_new):
                        print("New File Created")
                        df_coin_all.to_csv(csv_all_coins_full_new, mode='w')
                    else:
                        print("Appended")
                        df_coin_all.to_csv(csv_all_coins_full_new, mode='a', header=False)
                    number_of_coins = number_of_coins +1
    
            except Exception as e:
                logging.error(e)
                # logging.debug("Could not update data for {curr} from {exchange}".format(curr=symbol, exchange=exchange))
                not_updated[exchange].append(symbol)
    
    logging.error("Did not update the following. Try again.\n {not_updated}".format(not_updated=not_updated))
    print(number_of_coins)
    print(coin_exchange_combination_to_delete)
    delete_coins_from_CSV(coin_exchange_combination_to_delete,df_csv_all_coins_full.reset_index(),csv_filename_read)

In [117]:
def delete_coins_from_CSV(coin_exchange_combination_to_delete,df_csv_all_coins_full,csv_filename_read):
    for exchange in coin_exchange_combination_to_delete:
        coins_list_to_delete = coin_exchange_combination_to_delete[exchange]
        for symbol in coins_list_to_delete:
            df_csv_all_coins_full = df_csv_all_coins_full[~((df_csv_all_coins_full['coin'] == symbol) & (df_csv_all_coins_full['exchange'] == exchange))] 
    df_csv_all_coins_full.set_index(['coin', 'exchange','unix_timestamp']).to_csv(csv_filename_read)

In [ ]:
coins_list_from_exchange = setupExchanges(list_of_exchanges)
update_and_delete_coin_exchange_combination(data_directory+'all_coins_day_full_1day.csv',data_directory+'all_coins_day_full_1day_new_coins.csv','1dayfull',coins_list_from_exchange)

In [ ]:
#coins_list = ["BTC"]
coins_list = [""]
setupExchanges(list_of_exchanges)
update_and_delete_coin_exchange_combination(data_directory+'all_coins_hour_full_1hour_.csv',data_directory+'all_coins_hour_full_1hour_.csv','1hour',coins_list)

In [118]:
# Maps csv (future data objects) to period granularity
# If we store all data together in a single data source, we'll change this to a function which returns corresponding rows
data_csv_period_mapping = {
    "1day": data_directory+'all_coins_day_full_1day.csv',
    "1hour": data_directory+'all_coins_hour_full_1hour.csv',
    "1min": data_directory+'all_coins_min_full_1min.csv',
    "1daycryptopia":data_directory+'all_coins_day_full_1day_Cryptopia.csv',
    '1daybtcbitfinex':data_directory+'BTC_Bitfinex_day_full_1day.csv',
    '1hourbtcbitfinex':data_directory+'BTC_Bitfinex_hour_full_1hour.csv'
}
frequency_resampling_period_mapping = {
    "day":'D',
    "hour":'H',
    "min":'M'
}
function_period_mapping = {
    '1day': ccw.get_historical_price_day,
    '1hour': ccw.get_historical_price_hour,
    '1min': ccw.get_historical_price_minute,
    '1dayfull' : ccw.get_historical_price_day_full,
    '1daycryptopia' : ccw.get_historical_price_day,
    '1daybtcbitfinex':ccw.get_historical_price_day,
    '1hourbtcbitfinex':ccw.get_historical_price_hour
}

In [119]:
#df_csv = pd.read_csv('all_coins_day_full.csv', index_col=None)
indicator_list = ['unix_timestamp','BBANDS_BANDWIDTH_PERCENT','MONEY_FLOW_INDEX',
                   'STOCH_PERCENT_K_MONEY_FLOW_INDEX','STOCH_PERCENT_D_MONEY_FLOW_INDEX','RSI','RSI_OVER_BOUGHT','RSI_OVER_SOLD',
                   'STOCHRSI_K','STOCHRSI_D','STOCH_PERCENT_K','STOCH_PERCENT_D','STOCH_OVER_BOUGHT','STOCH_OVER_SOLD','SMA_FAST','SMA_SLOW','SMA_TEST',
                  'MACD','MACD_SIGNAL','MACD_TEST','ON_BALANCE_VOLUME','ON_BALANCE_VOLUME_TEST']   

In [120]:
 # Technical Analysis Settings
EMA_FAST = 10
EMA_SLOW = 20
RSI_PERIOD = 14
RSI_OVER_BOUGHT = 70
RSI_OVER_SOLD = 30
RSI_AVG_PERIOD = 15
MACD_FAST = 12
MACD_SLOW = 26
MACD_SIGNAL = 9
STOCH_K = 14
STOCH_D = 3
STOCH_OVER_BOUGHT = 70
STOCH_OVER_SOLD = 30
from pyti import bollinger_bands
from pyti import money_flow_index
from pyti import stochastic
from pyti import simple_moving_average
from pyti import stochrsi
from pyti import on_balance_volume

In [121]:
def update_indicator(csv_filename,periods,timeframe,datetimeformat_string):
    """ Update the given csv_file with new column values for corr rows """
    df_csv = pd.read_csv(csv_filename, index_col=None,dayfirst=True)
    df_csv.drop_duplicates(subset=['coin','exchange','unix_timestamp'],inplace=True)
    #df_csv.time = df_csv.time.apply(lambda t: datetime.datetime.strptime(t, '%d-%m-%Y %H:%M:%S'))
    
    for indicator in indicator_list:
        if indicator not in df_csv.columns and indicator not in df_csv.index:
            df_csv[indicator] = np.nan
    #df_csv.unix_timestamp =  df_csv.time.apply(lambda t: time.mktime(datetime.datetime.strptime(str(t) ,'%d-%m-%Y %H:%M').timetuple()))
    df_csv = df_csv.set_index(['coin', 'exchange','unix_timestamp'])
    data = list(df_csv.index.get_level_values(0).unique())
    i=0
    j=0
    for coin_name in data:
        coin_df = df_csv[df_csv.index.get_level_values(0)==coin_name] 
        #print(coin_df)
        coin_df = coin_df.reset_index()
        coin_df = coin_df.sort_values(by=['exchange','unix_timestamp']).set_index(['coin', 'exchange','unix_timestamp'])
        #print(coin_df)
        df_groupby = coin_df.groupby(['exchange'], group_keys=False)
        for key, item in df_groupby:
            req_data = df_groupby.get_group(key)
            req_data2 = req_data.iloc[-periods:]

            start_date = req_data2.index.get_level_values(2)[0]
            end_date = req_data2.index.get_level_values(2)[req_data2.shape[0]-1]
            req_data2 = req_data[(req_data.index.get_level_values(2) >= start_date) & (req_data.index.get_level_values(2) <= end_date)]
            #print(req_data2)
            np_volumeto = np.array(req_data2.volumeto.values,dtype='f8')
            if len(np_volumeto)<20:
                j = j+1
                print(coin_name,j," Not Updated")
                continue
            req_data2['BBANDS_BANDWIDTH_PERCENT'] = pyti.bollinger_bands.percent_b(req_data2.close.values,20)
            req_data2['MONEY_FLOW_INDEX'] = money_flow_index.money_flow_index(req_data2.close.values, req_data2.high.values, req_data2.low.values, np_volumeto, 14)
            req_data2['STOCH_PERCENT_K_MONEY_FLOW_INDEX'] = pyti.stochastic.percent_k(req_data2.MONEY_FLOW_INDEX.values,14) * 100
            req_data2['STOCH_PERCENT_D_MONEY_FLOW_INDEX'] = pyti.simple_moving_average.simple_moving_average(req_data2.STOCH_PERCENT_K_MONEY_FLOW_INDEX.values,3)
            req_data2['RSI'] = talib.RSI(req_data2.close.values, timeperiod=RSI_PERIOD)
            req_data2['RSI_OVER_BOUGHT'] = np.where((req_data2.RSI >= RSI_OVER_BOUGHT) & (req_data2.RSI <= req_data2.RSI.shift(1)),1,0)
            req_data2['RSI_OVER_SOLD'] = np.where((req_data2.RSI <= RSI_OVER_SOLD) & (req_data2.RSI >= req_data2.RSI.shift(1)),1,0)
            req_data2['STOCHRSI_K'] = pyti.stochrsi.stochrsi(req_data2.close.values,14)
            req_data2['STOCHRSI_D'] = pyti.simple_moving_average.simple_moving_average(req_data2.STOCHRSI_K.values,3)
            req_data2['STOCH_PERCENT_K'] = pyti.stochastic.percent_k(req_data2.high.values,14) * 100
            req_data2['STOCH_PERCENT_D'] = pyti.simple_moving_average.simple_moving_average(req_data2.STOCH_PERCENT_K.values,3)
            req_data2['STOCH_OVER_BOUGHT'] = np.where((req_data2.STOCH_PERCENT_K >= STOCH_OVER_BOUGHT) & (req_data2.STOCH_PERCENT_K <= req_data2.STOCH_PERCENT_K.shift(1)),1,0)
            req_data2['STOCH_OVER_SOLD'] = np.where((req_data2.STOCH_PERCENT_K <= STOCH_OVER_SOLD) & (req_data2.STOCH_PERCENT_K >= req_data2.STOCH_PERCENT_K.shift(1)),1,0)
            req_data2['SMA_FAST'] = talib.SMA(req_data2.close.values,7)
            req_data2['SMA_SLOW'] = talib.SMA(req_data2.close.values,21)
            req_data2['SMA_TEST'] = np.where(req_data2.SMA_FAST>req_data2.SMA_SLOW,1,0)
            req_data2['ON_BALANCE_VOLUME'] = on_balance_volume.on_balance_volume(req_data2.close.values,np_volumeto)
            req_data2['ON_BALANCE_VOLUME_TEST'] = np.where(req_data2.ON_BALANCE_VOLUME>req_data2.ON_BALANCE_VOLUME.shift(1),1,0)
            """
            req_data2['Accumulation_Distribution_Oscillator'] = talib.ADOSC(req_data2.high.values,req_data2.low.values
                                                  ,req_data2.close.values,np_volumeto)
            req_data2['ADOSC_TEST'] = np.where((req_data2.Accumulation_Distribution_Oscillator>req_data2.Accumulation_Distribution_Oscillator.shift(1)) & (req_data2.Accumulation_Distribution_Oscillator>=0) & 
                                               (req_data2.Accumulation_Distribution_Oscillator.shift(1)<=0),1,0)
            """
            
            req_data2['MACD'],req_data2['MACD_SIGNAL'],MACD_HISTOGRAM= talib.MACD(req_data2.close.values,fastperiod=MACD_FAST,slowperiod=MACD_SLOW,signalperiod=MACD_SIGNAL)
            req_data2['MACD_TEST'] = np.where(req_data2.MACD>req_data2.MACD_SIGNAL,1,0)
            
            
            df_csv.update(req_data2)
            i = i+1
            print(coin_name,i)
            #print(df_csv.query('coin == @coin_name').tail(1))
            #sys.exit("Testing")
    df_csv.to_csv(csv_filename,date_format=datetimeStringformat_to_csv)
    print("Done")

In [ ]:
update_indicator(data_directory+'all_coins_day_full_1day.csv',250,'1day',datetimeStringformat_to_csv)
print("Done")  

In [156]:
update_indicator(data_directory+'all_coins_day_full_3days.csv',250,'3day',datetimeStringformat_to_csv)
print("Done")  

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

XWC 1


C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

XDN 2


C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

...

HSR 216
WTC 217
WTC 218


C:\Users\Administrator\Anaconda3\lib\site-packages\pyti\stochastic.py:21: RuntimeWarning: invalid value encountered in double_scalars
  np.min(data[idx+1-period:idx+1]))) for idx in range(period-1, len(data))]
C:\Users\Administrator\Anaconda3\lib\site-packages\pyti\relative_strength_index.py:35: RuntimeWarning: divide by zero encountered in double_scalars
  rsi.append(100 - (100 / (1 + rs)))
C:\Users\Administrator\Anaconda3\lib\site-packages\pyti\relative_strength_index.py:47: RuntimeWarning: divide by zero encountered in double_scalars
  rsi.append(100 - (100 / (1 + rs)))
C:\Users\Administrator\Anaconda3\lib\site-packages\pyti\stochrsi.py:16: RuntimeWarning: invalid value encountered in double_scalars
  stochrsi = [100 * ((rsi[idx] - np.min(rsi[idx+1-period:idx+1])) / (np.max(rsi[idx+1-period:idx+1]) - np.min(rsi[idx+1-period:idx+1]))) for idx in range(period-1, len(rsi))]
C:\Users\Administrator\Anaconda3\lib\site-packages\pyti\stochastic.py:21: RuntimeWarning: invalid value encounter

LRC 219
LRC 220
YOYOW 221
BQX 222
FUN 223
KNC 224
KNC 225
LINK 226
SNGLS 227
SNM 228
SNM 229
ZRX 230
ZRX 231
ADA 232
ADA 233
EOS 234
EOS 235
MDA 236
MTL 237
SALT 238
SALT 239
SUB 240
SUB 241
MANA 242
MANA 243
MANA 244
DNT 245
DNT 246
TIX 247
AST 248
BTG 249
BTG 250
BTG 251
ENG 252
ENG 253
ENJ 254
ENJ 255
EVX 256
ICN 257
MOD 258
MOD 259
MTH 260
MTH 261
OAX 262
POWR 263
POWR 264
POWR 265
RCN 266
RCN 267
REQ 268
REQ 269
TRX 270
TRX 271
VIB 272
VIB 273
MER 274
AMB 275
AMB 276
ARN 277
ARN 278
BCPT 279
BCPT 280
BCPT 281
DLT 282
NULS 283
NULS 284
VET 285
VET 286
QSP 287
QSP 288
BCD 289
BCD 290
CAG 291
GVT 292
GVT 293
HST 294
KCS 295
NEBL 296
NEBL 297
PPT 298
PPT 299
PURA 300
RPX 301
RPX 302
UKG 303
UKG 304
XAS 305
BTS 306
CDT 307
CND 308
DGD 309
FUEL 310
GXS 311
LEND 312
LEND 313
POE 314
TNT 315
WABI 316
AION 317
AION 318
BNTY 319
BRD 320
BRD 321
DBC 322
DENT 2  Not Updated
DENT 323
DRGN 324
EDO 325
ELF 326
ELF 327
ELIX 328
FLIXX 329
GTO 330
GTO 331
ICX 332
LA 333
ONION 334
OST 335
PBL 336
PO

C:\Users\Administrator\Anaconda3\lib\site-packages\pyti\relative_strength_index.py:35: RuntimeWarning: divide by zero encountered in double_scalars
  rsi.append(100 - (100 / (1 + rs)))
C:\Users\Administrator\Anaconda3\lib\site-packages\pyti\relative_strength_index.py:47: RuntimeWarning: divide by zero encountered in double_scalars
  rsi.append(100 - (100 / (1 + rs)))


SOUL 458
MAN 459
OPEN 460
LALA 461
CHP 462
TIME 463
SHL 464
APH 465
ABT 466
BAX 467
BKX 4  Not Updated
CBC 5  Not Updated
CBC 6  Not Updated
CMCT 7  Not Updated
NLC2 8  Not Updated
NPXS 9  Not Updated
NAS 10  Not Updated
MFT 11  Not Updated
AOA 12  Not Updated
TFD 13  Not Updated
GO 14  Not Updated
TMT 15  Not Updated
OMX 16  Not Updated
DATX 17  Not Updated
ZINC 18  Not Updated
Done
Done


In [155]:
update_indicator(data_directory+'all_coins_day_full_7days.csv',250,'7day',datetimeStringformat_to_csv)
print("Done")  

XWC 1
XDN 2
EMC2 3


C:\Users\Administrator\Anaconda3\lib\site-packages\pyti\stochrsi.py:16: RuntimeWarning: invalid value encountered in double_scalars
  stochrsi = [100 * ((rsi[idx] - np.min(rsi[idx+1-period:idx+1])) / (np.max(rsi[idx+1-period:idx+1]) - np.min(rsi[idx+1-period:idx+1]))) for idx in range(period-1, len(rsi))]
C:\Users\Administrator\Anaconda3\lib\site-packages\pyti\stochastic.py:21: RuntimeWarning: invalid value encountered in double_scalars
  np.min(data[idx+1-period:idx+1]))) for idx in range(period-1, len(data))]


THC 4
VTR 5
CURE 6
ENRG 7
KORE 8
PINK 9
RBY 10
RDD 11
ABY 12
BAY 13
BLOCK 14
CLOAK 1  Not Updated
CLOAK 15
DOPE 16
FTC 17
IOC 18
MONA 19
SYS 2  Not Updated
SYS 20
VRC 21
XLM 22
XLM 23
XMG 24
XMY 25
BLK 26
BURST 27
BYC 28
CANN 29
DGB 30
DGB 31
DOGE 32
EFL 33
EXCL 34
LTC 35
LTC 36
LTC 37
NAV 38
NAV 39
NEOS 40
NLG 41
NXT 42
POT 43
PTC 44
SLR 45
SPR 46
TRUST 47
VIA 48
VIA 49
VTC 50
XMR 51
XMR 52
XRP 53
XRP 54
XST 55
XCP 56
BITB 57
GEO 58
FLDC 59
GRC 60
FLO 61
DASH 62
DASH 63
DASH 64
CLAM 65
NBT 66
XEM 3  Not Updated
XEM 67
DMD 68
MUE 69
GAM 70
SPHR 71
OK 72
PPC 73
ERC 74
AEON 75
ETH 76
ETH 77
ETH 78
TX 79
BCY 80
EXP 81
OMNI 82
AMP 83
AUR 84
BNT 85
BNT 86


C:\Users\Administrator\Anaconda3\lib\site-packages\pyti\stochastic.py:21: RuntimeWarning: invalid value encountered in double_scalars
  np.min(data[idx+1-period:idx+1]))) for idx in range(period-1, len(data))]


RVR 87
EMC 88
FCT 89
EGC 90
SLS 91
GRS 4  Not Updated
GRS 92
RADS 93
DCR 94
XVG 95
XVG 96
MEME 97
GAME 98
STEEM 99
STEEM 100
2GIVE 101
LSK 102
LSK 103
BRK 104
WAVES 105
WAVES 106
LBC 107
BRX 108
ETC 109
ETC 110
ETC 111
STRAT 112
STRAT 113
COVAL 114


C:\Users\Administrator\Anaconda3\lib\site-packages\pyti\stochrsi.py:16: RuntimeWarning: invalid value encountered in double_scalars
  stochrsi = [100 * ((rsi[idx] - np.min(rsi[idx+1-period:idx+1])) / (np.max(rsi[idx+1-period:idx+1]) - np.min(rsi[idx+1-period:idx+1]))) for idx in range(period-1, len(rsi))]


GLD 115
SYNX 116
UNB 117
EBST 118
VRM 119
NXS 5  Not Updated
NXS 120
REP 6  Not Updated
REP 121
SHIFT 122
ARDR 123
XZC 124
XZC 125
NEO 126
NEO 127
NEO 128
ZEC 129
ZEC 130
ZCL 131
IOP 132
GOLOS 133
UBQ 134
GBG 135
KMD 136
KMD 137
PIVX 138
PIVX 139
ION 140
LMC 141
SIB 142
QWARK 143
CRW 144
SWT 145
MLN 146
SEQ 147
ARK 148
ARK 149
DYN 150
TKS 151
MUSIC 152
DTB 153
INCNT 154
GBYTE 155
GNT 7  Not Updated
GNT 156
NXC 157
EDG 158
RLC 159
RLC 160
WINGS 161
WINGS 162
GUP 163
GNO 164
LUN 165
LUN 166
HMQ 167
ANT 168
SC 8  Not Updated
SC 169
BAT 170
BAT 171
ZEN 9  Not Updated
ZEN 172
QRL 173
PTOY 174
CFI 175
NMR 176
SNT 177
SNT 178
SNT 179
DCT 180
MCO 181
MCO 182
XEL 183
ADT 184
PAY 185
PAY 186
ADX 187
ADX 188
STORJ 189
STORJ 190
BNB 191
CVC 10  Not Updated
CVC 192
CVC 193
OMG 194
OMG 195
OMG 196
PART 197
QTUM 198
QTUM 199
QTUM 200
BCH 201
BCH 202
BCH 203
GAS 204
GAS 205
HSR 206
HSR 207
WTC 208
WTC 209
LRC 210
LRC 211
YOYOW 212
BQX 213
FUN 214
KNC 215
KNC 216
LINK 217
SNGLS 218
SNM 219
SNM 220
ZRX 

In [154]:
update_indicator(data_directory+'all_coins_day_full_10days.csv',250,'10day',datetimeStringformat_to_csv)
print("Done")  

XWC 1
XDN 2
EMC2 3


C:\Users\Administrator\Anaconda3\lib\site-packages\pyti\stochrsi.py:16: RuntimeWarning: invalid value encountered in double_scalars
  stochrsi = [100 * ((rsi[idx] - np.min(rsi[idx+1-period:idx+1])) / (np.max(rsi[idx+1-period:idx+1]) - np.min(rsi[idx+1-period:idx+1]))) for idx in range(period-1, len(rsi))]
C:\Users\Administrator\Anaconda3\lib\site-packages\pyti\stochastic.py:21: RuntimeWarning: invalid value encountered in double_scalars
  np.min(data[idx+1-period:idx+1]))) for idx in range(period-1, len(data))]


THC 4
VTR 5
CURE 6
ENRG 7
KORE 8
PINK 9
RBY 10
RDD 11
ABY 12
BAY 13
BLOCK 14
CLOAK 1  Not Updated
CLOAK 15
DOPE 16
FTC 17
IOC 18
MONA 19
SYS 2  Not Updated
SYS 20
VRC 21
XLM 22
XLM 23
XMG 24
XMY 25
BLK 26
BURST 27
BYC 28
CANN 29
DGB 30
DGB 31
DOGE 32
EFL 33
EXCL 34
LTC 35
LTC 36
LTC 37
NAV 38
NAV 39
NEOS 40
NLG 41
NXT 42
POT 43
PTC 44
SLR 45
SPR 46
TRUST 47
VIA 3  Not Updated
VIA 48
VTC 49
XMR 50
XMR 51
XRP 52
XRP 53
XST 54
XCP 55
BITB 56
GEO 57
FLDC 58
GRC 59
FLO 60
DASH 61
DASH 62
DASH 63
CLAM 64
NBT 65
XEM 4  Not Updated
XEM 66
DMD 67
MUE 68
GAM 69
SPHR 70
OK 71
PPC 72
ERC 73
AEON 74
ETH 75
ETH 76
ETH 77
TX 78
BCY 79
EXP 80
OMNI 81
AMP 82
AUR 83
BNT 84

C:\Users\Administrator\Anaconda3\lib\site-packages\pyti\stochastic.py:21: RuntimeWarning: invalid value encountered in double_scalars
  np.min(data[idx+1-period:idx+1]))) for idx in range(period-1, len(data))]



BNT 85
RVR 86
EMC 87
FCT 88
EGC 89
SLS 90
GRS 5  Not Updated
GRS 91
RADS 92
DCR 93
XVG 94
XVG 95
MEME 96
GAME 97
STEEM 6  Not Updated
STEEM 98
2GIVE 99
LSK 100
LSK 101
BRK 102
WAVES 103
WAVES 104
LBC 105
BRX 106
ETC 107
ETC 108
ETC 109
STRAT 110
STRAT 111
COVAL 112
GLD 113
SYNX 114
UNB 115
EBST 116
VRM 117
NXS 7  Not Updated
NXS 118
REP 8  Not Updated
REP 119
SHIFT 120
ARDR 121
XZC 122
XZC 123
NEO 124
NEO 125
NEO 126
ZEC 127
ZEC 128
ZCL 129
IOP 130
GOLOS 131
UBQ 132
GBG 133
KMD 134
KMD 135
PIVX 9  Not Updated
PIVX 136
ION 137
LMC 138
SIB 139
QWARK 140
CRW 141
SWT 142
MLN 143
SEQ 144
ARK 145
ARK 146
DYN 147
TKS 148
MUSIC 149
DTB 150
INCNT 151
GBYTE 152
GNT 10  Not Updated
GNT 153
NXC 154
EDG 155
RLC 11  Not Updated
RLC 156
WINGS 157
WINGS 158
GUP 159
GNO 160
LUN 161
LUN 162
HMQ 163
ANT 164
SC 12  Not Updated
SC 165
BAT 166
BAT 167
ZEN 13  Not Updated
ZEN 168
QRL 169
PTOY 170
CFI 171
NMR 172
SNT 173
SNT 174
SNT 175
DCT 176
MCO 177
MCO 178
XEL 179
ADT 180
PAY 181
PAY 182
ADX 183
ADX 184


In [153]:
update_indicator(data_directory+'all_coins_day_full_14days.csv',250,'14day',datetimeStringformat_to_csv)
print("Done")  

XWC 1
XDN 2
EMC2 3


C:\Users\Administrator\Anaconda3\lib\site-packages\pyti\stochastic.py:21: RuntimeWarning: invalid value encountered in double_scalars
  np.min(data[idx+1-period:idx+1]))) for idx in range(period-1, len(data))]


THC 4
VTR 5
CURE 6
ENRG 7
KORE 8
PINK 9
RBY 10
RDD 11
ABY 12
BAY 13
BLOCK 14
CLOAK 1  Not Updated
CLOAK 15
DOPE 16
FTC 17
IOC 18
MONA 19
SYS 2  Not Updated
SYS 20
VRC 21
XLM 3  Not Updated
XLM 22
XMG 23
XMY 24
BLK 25
BURST 26
BYC 27
CANN 28
DGB 29
DGB 4  Not Updated
DOGE 30
EFL 31
EXCL 32
LTC 33
LTC 34
LTC 5  Not Updated
NAV 6  Not Updated
NAV 35
NEOS 36
NLG 37
NXT 38
POT 39
PTC 40
SLR 41
SPR 42
TRUST 43
VIA 7  Not Updated
VIA 44
VTC 45
XMR 46
XMR 47
XRP 48
XRP 49
XST 50
XCP 51
BITB 52
GEO 53
FLDC 54
GRC 55
FLO 56
DASH 57
DASH 58
DASH 8  Not Updated
CLAM 59
NBT 60
XEM 9  Not Updated
XEM 61
DMD 62
MUE 63
GAM 64
SPHR 65
OK 66
PPC 67
ERC 68
AEON 69
ETH 70
ETH 71
ETH 10  Not Updated
TX 72
BCY 73
EXP 74
OMNI 75
AMP 76
AUR 77
BNT 78


C:\Users\Administrator\Anaconda3\lib\site-packages\pyti\stochastic.py:21: RuntimeWarning: invalid value encountered in double_scalars
  np.min(data[idx+1-period:idx+1]))) for idx in range(period-1, len(data))]


BNT 79
RVR 80
EMC 81
FCT 82
EGC 83
SLS 84
GRS 11  Not Updated
GRS 85
RADS 86
DCR 87
XVG 88
XVG 89
MEME 90
GAME 91
STEEM 12  Not Updated
STEEM 92
2GIVE 93
LSK 13  Not Updated
LSK 94
BRK 95
WAVES 14  Not Updated
WAVES 96
LBC 97
BRX 98
ETC 99
ETC 100
ETC 15  Not Updated
STRAT 101
STRAT 102
COVAL 103
GLD 104
SYNX 105
UNB 106
EBST 107
VRM 108
NXS 16  Not Updated
NXS 109
REP 17  Not Updated
REP 110
SHIFT 111
ARDR 112
XZC 18  Not Updated
XZC 113
NEO 114
NEO 115
NEO 19  Not Updated
ZEC 116
ZEC 117
ZCL 118
IOP 119
GOLOS 120
UBQ 121
GBG 122
KMD 123
KMD 124
PIVX 20  Not Updated
PIVX 125
ION 126
LMC 127
SIB 128
QWARK 129
CRW 130
SWT 131
MLN 132
SEQ 133
ARK 134
ARK 135
DYN 136
TKS 137
MUSIC 138
DTB 139
INCNT 140
GBYTE 141
GNT 21  Not Updated
GNT 142
NXC 143
EDG 144
RLC 22  Not Updated
RLC 145
WINGS 23  Not Updated
WINGS 146
GUP 147
GNO 148
LUN 24  Not Updated
LUN 149
HMQ 150
ANT 151
SC 25  Not Updated
SC 152
BAT 153
BAT 154
ZEN 26  Not Updated
ZEN 155
QRL 156
PTOY 157
CFI 158
NMR 159
SNT 160
SNT 16

In [ ]:
update_indicator(data_directory+'all_coins_day_full_1day_Cryptopia.csv',250,'1day',datetimeStringformat_to_csv)
print("Done")  

In [ ]:
update_indicator(data_directory+'all_coins_day_full_3days_Cryptopia.csv',250,'3day',datetimeStringformat_to_csv)
print("Done")

In [ ]:
update_indicator(data_directory+'all_coins_day_full_7days_Cryptopia.csv',250,'7day',datetimeStringformat_to_csv)
print("Done")

In [ ]:
update_indicator(data_directory+'all_coins_day_full_10days_Cryptopia.csv',250,'10day',datetimeStringformat_to_csv)
print("Done")

In [ ]:
update_indicator(data_directory+'all_coins_day_full_14days_Cryptopia.csv',250,'14day',datetimeStringformat_to_csv)
print("Done")

In [ ]:
update_indicator(data_directory+'all_coins_min_full_1min.csv',250,'1min')
print("Done")

In [122]:
def resample(csv_filename,period,resampling_multiplier,exchange,datetimeformat_string,output_file_name):
    df_csv = pd.read_csv(csv_filename,dayfirst=True)
    #df_csv['time'] = df_csv.unix_timestamp.apply(lambda t: datetime.datetime.fromtimestamp(int(t)).strftime(datetimeformat_string))
    df_csv.unix_timestamp = pd.to_datetime(df_csv.unix_timestamp,unit="s",utc=True)
    #df_csv['time'].to_csv('time.csv')
    df_csv = df_csv.reset_index()
    #print(df_csv.columns)
    for indicator in indicator_list:
        if indicator not in df_csv.columns:
            df_csv[indicator] = np.nan
    df_csv = df_csv.set_index(['coin', 'exchange','unix_timestamp'])
    data = list(df_csv.index.get_level_values(0).unique())
    i=0
    all_dataframes = []
    resampling_period = ""+str(resampling_multiplier)+frequency_resampling_period_mapping[period]
    output_csv_filename = output_file_name
    for coin_name in data:
        coin_df = df_csv[df_csv.index.get_level_values(0)==coin_name] 
        coin_df = coin_df.reset_index()
        coin_df = coin_df.sort_values(by=['exchange','unix_timestamp']).set_index(['coin', 'exchange','unix_timestamp'])
        #print(coin_df)
        df_groupby = coin_df.groupby(['exchange'], group_keys=False)
        for key, item in df_groupby:
            req_data = df_groupby.get_group(key)
            #print(type(req_data.index))
            req_data = req_data.resample(resampling_period,level=2,closed='right',label='right').agg({'open': 'first', 
                                     'high': 'max', 
                                     'low': 'min', 
                                     'close': 'last',
                                    'volumeto':'sum',
                                        'volumefrom':'sum'})

            req_data['coin'] = coin_name
            req_data['exchange'] = key
            
            #print(req_data)
            req_data = req_data.reset_index()
            req_data['unix_timestamp'] =  req_data.unix_timestamp.apply(lambda t: time.mktime(datetime.datetime.strptime(str(t), '%Y-%m-%d %H:%M:%S').timetuple()))
            #req_data['time'].to_csv("Req_data_Time.csv")
            req_data['time'] = pd.to_datetime(req_data.unix_timestamp,unit="s",utc=True)

            req_data = req_data.set_index(['coin','exchange','unix_timestamp'])
            i = i+1
            print(coin_name,i)
            #req_data = req_data.drop(labels=['time'],axis=1)
            all_dataframes.append(req_data)
    dataframe_answer = pd.concat(all_dataframes)
    #dataframe_answer['time'].to_csv('Time.csv')
    dataframe_answer.to_csv(output_csv_filename,date_format=datetimeStringformat_to_csv)
    print("Done")

In [152]:
resample(data_csv_period_mapping['1day'],'day',3,"",datetimeStringformat_to_csv,data_directory+'all_coins_day_full_3days.csv')

XWC 1
XDN 2
EMC2 3
THC 4
VTR 5
CURE 6
ENRG 7
KORE 8
PINK 9
RBY 10
RDD 11
ABY 12
BAY 13
BLOCK 14
CLOAK 15
CLOAK 16
DOPE 17
FTC 18
IOC 19
MONA 20
SYS 21
SYS 22
VRC 23
XLM 24
XLM 25
XMG 26
XMY 27
BLK 28
BURST 29
BYC 30
CANN 31
DGB 32
DGB 33
DOGE 34
EFL 35
EXCL 36
LTC 37
LTC 38
LTC 39
NAV 40
NAV 41
NEOS 42
NLG 43
NXT 44
POT 45
PTC 46
SLR 47
SPR 48
TRUST 49
VIA 50
VIA 51
VTC 52
XMR 53
XMR 54
XRP 55
XRP 56
XST 57
XCP 58
BITB 59
GEO 60
FLDC 61
GRC 62
FLO 63
DASH 64
DASH 65
DASH 66
CLAM 67
NBT 68
XEM 69
XEM 70
DMD 71
MUE 72
GAM 73
SPHR 74
OK 75
PPC 76
ERC 77
AEON 78
ETH 79
ETH 80
ETH 81
TX 82
BCY 83
EXP 84
OMNI 85
AMP 86
AUR 87
BNT 88
BNT 89
RVR 90
EMC 91
FCT 92
EGC 93
SLS 94
GRS 95
GRS 96
RADS 97
DCR 98
XVG 99
XVG 100
MEME 101
GAME 102
STEEM 103
STEEM 104
2GIVE 105
LSK 106
LSK 107
BRK 108
WAVES 109
WAVES 110
LBC 111
BRX 112
ETC 113
ETC 114
ETC 115
STRAT 116
STRAT 117
COVAL 118
GLD 119
SYNX 120
UNB 121
EBST 122
VRM 123
NXS 124
NXS 125
REP 126
REP 127
SHIFT 128
ARDR 129
XZC 130
XZC 131
NEO 132


In [151]:
resample(data_csv_period_mapping['1day'],'day',7,"",datetimeStringformat_to_csv,data_directory+'all_coins_day_full_7days.csv')

XWC 1
XDN 2
EMC2 3
THC 4
VTR 5
CURE 6
ENRG 7
KORE 8
PINK 9
RBY 10
RDD 11
ABY 12
BAY 13
BLOCK 14
CLOAK 15
CLOAK 16
DOPE 17
FTC 18
IOC 19
MONA 20
SYS 21
SYS 22
VRC 23
XLM 24
XLM 25
XMG 26
XMY 27
BLK 28
BURST 29
BYC 30
CANN 31
DGB 32
DGB 33
DOGE 34
EFL 35
EXCL 36
LTC 37
LTC 38
LTC 39
NAV 40
NAV 41
NEOS 42
NLG 43
NXT 44
POT 45
PTC 46
SLR 47
SPR 48
TRUST 49
VIA 50
VIA 51
VTC 52
XMR 53
XMR 54
XRP 55
XRP 56
XST 57
XCP 58
BITB 59
GEO 60
FLDC 61
GRC 62
FLO 63
DASH 64
DASH 65
DASH 66
CLAM 67
NBT 68
XEM 69
XEM 70
DMD 71
MUE 72
GAM 73
SPHR 74
OK 75
PPC 76
ERC 77
AEON 78
ETH 79
ETH 80
ETH 81
TX 82
BCY 83
EXP 84
OMNI 85
AMP 86
AUR 87
BNT 88
BNT 89
RVR 90
EMC 91
FCT 92
EGC 93
SLS 94
GRS 95
GRS 96
RADS 97
DCR 98
XVG 99
XVG 100
MEME 101
GAME 102
STEEM 103
STEEM 104
2GIVE 105
LSK 106
LSK 107
BRK 108
WAVES 109
WAVES 110
LBC 111
BRX 112
ETC 113
ETC 114
ETC 115
STRAT 116
STRAT 117
COVAL 118
GLD 119
SYNX 120
UNB 121
EBST 122
VRM 123
NXS 124
NXS 125
REP 126
REP 127
SHIFT 128
ARDR 129
XZC 130
XZC 131
NEO 132


In [150]:
resample(data_csv_period_mapping['1day'],'day',10,"",datetimeStringformat_to_csv,data_directory+'all_coins_day_full_10days.csv')

XWC 1
XDN 2
EMC2 3
THC 4
VTR 5
CURE 6
ENRG 7
KORE 8
PINK 9
RBY 10
RDD 11
ABY 12
BAY 13
BLOCK 14
CLOAK 15
CLOAK 16
DOPE 17
FTC 18
IOC 19
MONA 20
SYS 21
SYS 22
VRC 23
XLM 24
XLM 25
XMG 26
XMY 27
BLK 28
BURST 29
BYC 30
CANN 31
DGB 32
DGB 33
DOGE 34
EFL 35
EXCL 36
LTC 37
LTC 38
LTC 39
NAV 40
NAV 41
NEOS 42
NLG 43
NXT 44
POT 45
PTC 46
SLR 47
SPR 48
TRUST 49
VIA 50
VIA 51
VTC 52
XMR 53
XMR 54
XRP 55
XRP 56
XST 57
XCP 58
BITB 59
GEO 60
FLDC 61
GRC 62
FLO 63
DASH 64
DASH 65
DASH 66
CLAM 67
NBT 68
XEM 69
XEM 70
DMD 71
MUE 72
GAM 73
SPHR 74
OK 75
PPC 76
ERC 77
AEON 78
ETH 79
ETH 80
ETH 81
TX 82
BCY 83
EXP 84
OMNI 85
AMP 86
AUR 87
BNT 88
BNT 89
RVR 90
EMC 91
FCT 92
EGC 93
SLS 94
GRS 95
GRS 96
RADS 97
DCR 98
XVG 99
XVG 100
MEME 101
GAME 102
STEEM 103
STEEM 104
2GIVE 105
LSK 106
LSK 107
BRK 108
WAVES 109
WAVES 110
LBC 111
BRX 112
ETC 113
ETC 114
ETC 115
STRAT 116
STRAT 117
COVAL 118
GLD 119
SYNX 120
UNB 121
EBST 122
VRM 123
NXS 124
NXS 125
REP 126
REP 127
SHIFT 128
ARDR 129
XZC 130
XZC 131
NEO 132


In [149]:
resample(data_csv_period_mapping['1day'],'day',14,"",datetimeStringformat_to_csv,data_directory+'all_coins_day_full_14days.csv')

XWC 1
XDN 2
EMC2 3
THC 4
VTR 5
CURE 6
ENRG 7
KORE 8
PINK 9
RBY 10
RDD 11
ABY 12
BAY 13
BLOCK 14
CLOAK 15
CLOAK 16
DOPE 17
FTC 18
IOC 19
MONA 20
SYS 21
SYS 22
VRC 23
XLM 24
XLM 25
XMG 26
XMY 27
BLK 28
BURST 29
BYC 30
CANN 31
DGB 32
DGB 33
DOGE 34
EFL 35
EXCL 36
LTC 37
LTC 38
LTC 39
NAV 40
NAV 41
NEOS 42
NLG 43
NXT 44
POT 45
PTC 46
SLR 47
SPR 48
TRUST 49
VIA 50
VIA 51
VTC 52
XMR 53
XMR 54
XRP 55
XRP 56
XST 57
XCP 58
BITB 59
GEO 60
FLDC 61
GRC 62
FLO 63
DASH 64
DASH 65
DASH 66
CLAM 67
NBT 68
XEM 69
XEM 70
DMD 71
MUE 72
GAM 73
SPHR 74
OK 75
PPC 76
ERC 77
AEON 78
ETH 79
ETH 80
ETH 81
TX 82
BCY 83
EXP 84
OMNI 85
AMP 86
AUR 87
BNT 88
BNT 89
RVR 90
EMC 91
FCT 92
EGC 93
SLS 94
GRS 95
GRS 96
RADS 97
DCR 98
XVG 99
XVG 100
MEME 101
GAME 102
STEEM 103
STEEM 104
2GIVE 105
LSK 106
LSK 107
BRK 108
WAVES 109
WAVES 110
LBC 111
BRX 112
ETC 113
ETC 114
ETC 115
STRAT 116
STRAT 117
COVAL 118
GLD 119
SYNX 120
UNB 121
EBST 122
VRM 123
NXS 124
NXS 125
REP 126
REP 127
SHIFT 128
ARDR 129
XZC 130
XZC 131
NEO 132


In [ ]:
resample(data_csv_period_mapping['1daycryptopia'],'day',3,"Cryptopia",datetimeStringformat_to_csv,data_directory+'all_coins_day_full_3days_Cryptopia.csv')

In [ ]:
resample(data_csv_period_mapping['1daycryptopia'],'day',7,"Cryptopia",datetimeStringformat_to_csv,data_directory+'all_coins_day_full_7days_Cryptopia.csv')

In [ ]:
resample(data_csv_period_mapping['1daycryptopia'],'day',10,"Cryptopia",datetimeStringformat_to_csv,data_directory+'all_coins_day_full_10days_Cryptopia.csv')

In [ ]:
resample(data_csv_period_mapping['1daycryptopia'],'day',14,"Cryptopia",datetimeStringformat_to_csv,data_directory+'all_coins_day_full_14days_Cryptopia.csv')

In [135]:
resample(data_csv_period_mapping['1hourbtcbitfinex'],'hour',4,"Bitfinex",datetimeStringformat_to_csv,data_directory+'BTC_Bitfinex_hour_full_4hours.csv')

BTC 1
Done


In [134]:
resample(data_csv_period_mapping['1hourbtcbitfinex'],'hour',6,"Bitfinex",datetimeStringformat_to_csv,data_directory+'BTC_Bitfinex_hour_full_6hours.csv')

BTC 1
Done


In [133]:
resample(data_csv_period_mapping['1hourbtcbitfinex'],'hour',12,"Bitfinex",datetimeStringformat_to_csv,data_directory+'BTC_Bitfinex_hour_full_12hours.csv')

BTC 1
Done


In [132]:
resample(data_csv_period_mapping['1daybtcbitfinex'],'day',3,"Bitfinex",datetimeStringformat_to_csv,data_directory+'BTC_Bitfinex_day_full_3days.csv')

BTC 1
Done


In [131]:
resample(data_csv_period_mapping['1daybtcbitfinex'],'day',7,"Bitfinex",datetimeStringformat_to_csv,data_directory+'BTC_Bitfinex_day_full_7days.csv')

BTC 1
Done


In [130]:
resample(data_csv_period_mapping['1daybtcbitfinex'],'day',14,"Bitfinex",datetimeStringformat_to_csv,data_directory+'BTC_Bitfinex_day_full_14days.csv')

BTC 1
Done


In [123]:
def fetch_data_api(coin=COIN, to_curr=CURR, nperiods=1, period='1day',exchange_name=EXCHANGES[0]):
    """ Fetch data for coin over nperiods
        e.g. Get data for 'BTC' for past 12 hours in hours granularity
    """
    period = period.lower()
    func = function_period_mapping[period]
    if exchange_name == 'IDEX':
        to_curr = 'ETH'
    if coin == 'BTC':
        to_curr = 'USD'
    coin_last_nperiods = func(
        coin=coin,
        to_curr=to_curr,
        limit=nperiods,
        exchange=exchange_name
    )
    #print(coin_last_nperiods.index.get_level_values(0))
    #answer = coin_last_nperiods.iloc[-int(nperiods):]
    #print(answer.index.get_level_values(0))
    if coin_last_nperiods is not None:
        return coin_last_nperiods.iloc[-int(nperiods):]
    else:
        return None

In [124]:
def delete_latest_period_data(csv_filename,datetimeformat_string,exchanges):
    df_csv = pd.read_csv(csv_filename, index_col=None,dayfirst=True)
    #df_csv.time = df_csv.time.apply(lambda t: datetime.datetime.strptime(t, '%d-%m-%Y %H:%M'))
    #df_csv.unix_timestamp =  df_csv.time.apply(lambda t: time.mktime(datetime.datetime.strptime(str(t) ,'%d-%m-%Y %H:%M').timetuple()))
    df_csv = df_csv.set_index(['coin', 'exchange','unix_timestamp'])
    data = list(df_csv.index.get_level_values(0).unique())
    i=0
    j=0
    final_dataframe = []
    for coin_name in data:
        coin_df = df_csv[df_csv.index.get_level_values(0)==coin_name] 
        #print(coin_df)
        coin_df = coin_df.reset_index()
        coin_df = coin_df.sort_values(by=['exchange','unix_timestamp']).set_index(['coin', 'exchange','unix_timestamp'])
        #print(coin_df)
        df_groupby = coin_df.groupby(['exchange'], group_keys=False)
        for key, item in df_groupby:
            req_data = df_groupby.get_group(key)
            if len(exchanges) != 0 and key in exchanges:
                req_data = req_data[:-1]
                 #print(req_data.tail(1))
            final_dataframe.append(req_data)
    answer = pd.concat(final_dataframe).reset_index()
    #answer.to_csv('answer.csv')
    return answer

In [125]:
def update_csv_to_latest(period='1day',datetimeformat_string='%d-%m-%Y %H:%M',final_csv_column_order=[],exchanges=[]):
    """ Update the csv for given period upto current time for coin """
    period = period.lower()
    csv_filename = data_csv_period_mapping[period]  # Get corr csv
    #csv_filename = 'Experiment.csv'
    #df_coin_period = pd.read_csv(csv_filename)  # , index_col=['coin', 'exchange']
    df_coin_period = delete_latest_period_data(csv_filename,datetimeStringformat_to_csv,exchanges)
    #print(df_coin_period.tail(1)['time'])
    print("Updated Dataframe")
    csv_column_order = df_coin_period.columns.tolist()
    df_coin_period = df_coin_period.set_index(keys=['coin', 'exchange'])
    #df_coin_period.time = df_coin_period.time.apply(lambda t: datetime.datetime.strptime(t, '%d-%m-%Y %H:%M'))
    
    lst_new_data = []
    PRINT_MSG = "{:15} {!s:20} {!s:>20} {:>10}"
    logging.info(PRINT_MSG.format("Exchange", "Last Updated Time", "Elapsed Time", "nPeriodsAgo"))
    coins_in_csv = list(df_coin_period.index.get_level_values(0).unique())
    #coins_in_csv = ['ZRX']
    i=0
    j=0
    list_of_coin_exchange_combination_not_updated = []
    for coin in coins_in_csv:
        df_coin_period_coin = df_coin_period.loc[coin]
        # Group by exchange, sort on timestamp, and get the last row of that particular coin
        last_update = df_coin_period_coin.groupby('exchange', group_keys=False).apply(lambda c: c.sort_values(by='unix_timestamp').tail(1))
        logging.info("-" * 10 + " For coin - {}".format(coin))
        
        for exchange in last_update.index.values: #For every coin exchange combination
            if len(exchanges) != 0 and exchange in exchanges:
                last_updated_time = int(last_update.loc[exchange]['unix_timestamp']) #Get the time of the last row
                try:
                    # elapsed_time = datetime.datetime.now() - datetime.datetime.strptime(last_updated_time, '%Y-%m-%d %H:%M:%S')
                    # elapsed_time = datetime.datetime.now() - datetime.datetime.strptime(last_updated_time, '%d-%m-%Y %H:%M')
                    elapsed_time = int(time.time()) - last_updated_time
                except ValueError as e:
                    logging.info("Failed to parse time {} for {}--{}".format(last_updated_time, coin, exchange))
                    elapsed_time = datetime.datetime.now() - dateutil.parser.parse(last_updated_time)
                nperiods_ago = 0
                if period == '1day' or period == '1daycryptopia' or period =='1daybtcbitfinex':
                    nperiods_ago = elapsed_time/(60*60*24)
                elif period == '1hour' or period == '1hourbtcbitfinex':
                     nperiods_ago = elapsed_time/(60*60)
                """
                nperiods_ago = elapsed_time / datetime.timedelta(days=1 if period == '1day' or period == '1daycryptopia' or period =='1daybtcbitfinex' else 0,
                                                                 hours=1 if period == '1hour' or period == '1hourbtcbitfinex' else 0,
                                                                 minutes=1 if period == '1min' else 0,
                                                                 seconds=1)
                """
                nperiods_ago = np.floor(nperiods_ago)
                #print(nperiods_ago)
                logging.info(PRINT_MSG.format(exchange, last_updated_time, elapsed_time, nperiods_ago))
                #if coin == 'BTC':
                  #  nperiods_ago = nperiods_ago-1
                if nperiods_ago > 0:
                    logging.info("Updating data for {coin}-{exchange} from {last_updated_time}".format(
                        coin=coin, exchange=exchange, last_updated_time=last_updated_time)
                    )
                list_of_coin_exchange_combination_not_updated,lst_new_data,j,i = download_incremental_data(list_of_coin_exchange_combination_not_updated,lst_new_data,j,i,
                                                                                                          coin,exchange,nperiods_ago,period)
    #print("CSV column order "+str(csv_column_order))  
    return list_of_coin_exchange_combination_not_updated,lst_new_data,df_coin_period,csv_column_order,final_csv_column_order,csv_filename,datetimeStringformat_to_csv

In [126]:
def download_incremental_data(list_of_coin_exchange_combination_not_updated,lst_new_data,number_of_coin_exchange_combination_not_updated,
                              number_of_coin_exchange_combination_updated,coin,exchange,nperiods_ago,period):
    new_data_coin_period = fetch_data_api(
            coin=coin,
            nperiods=nperiods_ago,
            period=period,
            exchange_name=exchange
        )
    #time.sleep(3)
    #print(new_data_coin_period.shape)
    if new_data_coin_period is None:
        print(coin,exchange," Info Not available from API",str(number_of_coin_exchange_combination_not_updated))
        not_updated = {}
        not_updated['coin'] = coin
        not_updated['exchange'] = exchange
        not_updated['nperiods'] = nperiods_ago
        not_updated['period'] = period
        list_of_coin_exchange_combination_not_updated.append(not_updated)
        number_of_coin_exchange_combination_not_updated =number_of_coin_exchange_combination_not_updated+1
        return list_of_coin_exchange_combination_not_updated,lst_new_data,number_of_coin_exchange_combination_not_updated,number_of_coin_exchange_combination_updated
    new_data_coin_period['coin'] = coin
    new_data_coin_period['exchange'] = exchange
    new_data_coin_period = new_data_coin_period.reset_index()
    new_data_coin_period['time'] =  pd.to_datetime(new_data_coin_period.unix_timestamp,unit="s",utc=True)
    number_of_coin_exchange_combination_updated = number_of_coin_exchange_combination_updated + 1
    #print(coin,exchange,i)
    lst_new_data.append(new_data_coin_period)
    return list_of_coin_exchange_combination_not_updated,lst_new_data,number_of_coin_exchange_combination_not_updated,number_of_coin_exchange_combination_updated

In [127]:
def append_to_csv(lst_new_data,csv_column_order,final_csv_column_order,csv_filename,datetimeStringformat_to_csv,df_coin_period):
    df_new_data = pd.concat(lst_new_data)
    df_new_data = df_new_data.reset_index()
    curr_columns = df_new_data.columns.tolist()
    #print("Current columns "+str(curr_columns))
    df_coin_period = df_coin_period.reset_index()
    csv_column_order = df_coin_period.columns.tolist()
    column_order = [col for col in csv_column_order if col in curr_columns]
    #print("New Column Order "+str(column_order))
    df_new_data = df_new_data.reindex(columns=column_order)
    df_coin_period=df_coin_period.append(df_new_data)
    #df_new_data.to_csv(csv_filename, mode='a', header=False,index=False,date_format="%d-%m-%Y %H:%M:%S")
    df_coin_period = df_coin_period[final_csv_column_order]
    df_coin_period.drop_duplicates(subset=['coin','exchange','unix_timestamp'],inplace=True)
    df_coin_period.set_index(['coin', 'exchange','unix_timestamp']).to_csv(csv_filename,date_format=datetimeStringformat_to_csv)
    print("Done")
    #df_new_data.to_csv('Put it into CSV.csv')

In [148]:
columns_order = ['coin','exchange','unix_timestamp','time','open','high','low','close','volumefrom','volumeto',
                 'BBANDS_BANDWIDTH_PERCENT','MACD','MACD_SIGNAL','MACD_TEST','MONEY_FLOW_INDEX','ON_BALANCE_VOLUME','ON_BALANCE_VOLUME_TEST','RSI','RSI_OVER_BOUGHT',
                 'RSI_OVER_SOLD','SMA_FAST','SMA_SLOW','SMA_TEST','STOCHRSI_D','STOCHRSI_K','STOCH_OVER_BOUGHT','STOCH_OVER_SOLD','STOCH_PERCENT_D','STOCH_PERCENT_D_MONEY_FLOW_INDEX',
                 'STOCH_PERCENT_K','STOCH_PERCENT_K_MONEY_FLOW_INDEX']

list_of_coin_exchange_combination_not_updated,lst_new_data,df_coin_period,csv_column_order,final_csv_column_order,csv_filename,datetimeStringformat_to_csv = update_csv_to_latest('1day',
datetimeStringformat_to_csv,columns_order,['Binance','Bittrex','Kucoin'])
while list_of_coin_exchange_combination_not_updated is not None and len(list_of_coin_exchange_combination_not_updated) != 0:
    #print("Inside While loop")
    list_of_coin_exchange_combination_not_updated2 = []
    i,j=0,0
    for not_updated in list_of_coin_exchange_combination_not_updated:
        #print(not_updated)
        list_of_coin_exchange_combination_not_updated2,lst_new_data,j,i = download_incremental_data(list_of_coin_exchange_combination_not_updated2,lst_new_data,j,i,
                                coin=not_updated['coin'],
                              exchange=not_updated['exchange'],
                        nperiods_ago=not_updated['nperiods'],
                        period=not_updated['period']
                        )
    list_of_coin_exchange_combination_not_updated = list_of_coin_exchange_combination_not_updated2
append_to_csv(lst_new_data,csv_column_order,final_csv_column_order,csv_filename,datetimeStringformat_to_csv,df_coin_period)

2018-08-11 12:45:25,707 - INFO - Exchange        Last Updated Time            Elapsed Time nPeriodsAgo
2018-08-11 12:45:25,728 - INFO - ---------- For coin - XWC
2018-08-11 12:45:25,729 - INFO - Bittrex         1533168000                         803725        9.0
2018-08-11 12:45:25,730 - INFO - Updating data for XWC-Bittrex from 1533168000


Updated Dataframe


2018-08-11 12:45:26,493 - INFO - ---------- For coin - XDN
2018-08-11 12:45:26,495 - INFO - Bittrex         1533168000                         803726        9.0
2018-08-11 12:45:26,497 - INFO - Updating data for XDN-Bittrex from 1533168000
2018-08-11 12:45:27,247 - INFO - ---------- For coin - EMC2
2018-08-11 12:45:27,247 - INFO - Bittrex         1533168000                         803727        9.0
2018-08-11 12:45:27,247 - INFO - Updating data for EMC2-Bittrex from 1533168000
2018-08-11 12:45:28,002 - INFO - ---------- For coin - THC
2018-08-11 12:45:28,002 - INFO - Bittrex         1533168000                         803728        9.0
2018-08-11 12:45:28,002 - INFO - Updating data for THC-Bittrex from 1533168000
2018-08-11 12:45:28,733 - INFO - ---------- For coin - VTR
2018-08-11 12:45:28,734 - INFO - Bittrex         1533168000                         803728        9.0
2018-08-11 12:45:28,735 - INFO - Updating data for VTR-Bittrex from 1533168000
2018-08-11 12:45:29,508 - INFO - -----

Done


In [ ]:
columns_order = ['coin','exchange','unix_timestamp','time','open','high','low','close','volumefrom','volumeto',
                 'BBANDS_BANDWIDTH_PERCENT','MACD','MACD_SIGNAL','MACD_TEST','MONEY_FLOW_INDEX','ON_BALANCE_VOLUME','ON_BALANCE_VOLUME_TEST','RSI','RSI_OVER_BOUGHT',
                 'RSI_OVER_SOLD','SMA_FAST','SMA_SLOW','SMA_TEST','STOCHRSI_D','STOCHRSI_K','STOCH_OVER_BOUGHT','STOCH_OVER_SOLD','STOCH_PERCENT_D','STOCH_PERCENT_D_MONEY_FLOW_INDEX',
                 'STOCH_PERCENT_K','STOCH_PERCENT_K_MONEY_FLOW_INDEX']
list_of_coin_exchange_combination_not_updated,lst_new_data,df_coin_period,csv_column_order,final_csv_column_order,csv_filename,datetimeStringformat_to_csv = update_csv_to_latest('1dayCryptopia',datetimeStringformat_to_csv,columns_order,['Cryptopia'])
while len(list_of_coin_exchange_combination_not_updated) != 0:
    list_of_coin_exchange_combination_not_updated2 = []
    i,j=0,0
    for not_updated in list_of_coin_exchange_combination_not_updated:
        list_of_coin_exchange_combination_not_updated2,lst_new_data,j,i = download_incremental_data(list_of_coin_exchange_combination_not_updated2,lst_new_data,j,i,
                                coin=not_updated['coin'],
                              exchange=not_updated['exchange'],
                        nperiods_ago=not_updated['nperiods'],
                        period=not_updated['period']
                        )
    list_of_coin_exchange_combination_not_updated = list_of_coin_exchange_combination_not_updated2
append_to_csv(lst_new_data,csv_column_order,final_csv_column_order,csv_filename,datetimeStringformat_to_csv,df_coin_period)

In [128]:
columns_order = ['coin','exchange','unix_timestamp','time','open','high','low','close','volumefrom','volumeto',
                 'UPPER_BOLLINGER_BAND_VALUE','MIDDLE_BOLLINGER_BAND_VALUE','LOWER_BOLLINGER_BAND_VALUE']
list_of_coin_exchange_combination_not_updated,lst_new_data,df_coin_period,csv_column_order,final_csv_column_order,csv_filename,datetimeStringformat_to_csv = update_csv_to_latest('1daybtcbitfinex',datetimeStringformat_to_csv,columns_order,['Bitfinex'])
while len(list_of_coin_exchange_combination_not_updated) != 0:
    list_of_coin_exchange_combination_not_updated2 = []
    i,j=0,0
    for not_updated in list_of_coin_exchange_combination_not_updated:
        list_of_coin_exchange_combination_not_updated2,lst_new_data,j,i = download_incremental_data(list_of_coin_exchange_combination_not_updated2,lst_new_data,j,i,
                                coin=not_updated['coin'],
                              exchange=not_updated['exchange'],
                        nperiods_ago=not_updated['nperiods'],
                        period=not_updated['period']
                        )
    list_of_coin_exchange_combination_not_updated = list_of_coin_exchange_combination_not_updated2
append_to_csv(lst_new_data,csv_column_order,final_csv_column_order,csv_filename,datetimeStringformat_to_csv,df_coin_period)

2018-08-11 12:44:09,750 - INFO - Exchange        Last Updated Time            Elapsed Time nPeriodsAgo


Updated Dataframe


2018-08-11 12:44:10,015 - INFO - ---------- For coin - BTC
2018-08-11 12:44:10,062 - INFO - Bitfinex        1533600000                         371650        4.0
2018-08-11 12:44:10,064 - INFO - Updating data for BTC-Bitfinex from 1533600000


Done


In [129]:
columns_order = ['coin','exchange','unix_timestamp','time','open','high','low','close','volumefrom','volumeto',
                 'UPPER_BOLLINGER_BAND_VALUE','MIDDLE_BOLLINGER_BAND_VALUE','LOWER_BOLLINGER_BAND_VALUE']
list_of_coin_exchange_combination_not_updated,lst_new_data,df_coin_period,csv_column_order,final_csv_column_order,csv_filename,datetimeStringformat_to_csv = update_csv_to_latest('1hourbtcbitfinex',datetimeStringformat_to_csv,columns_order,['Bitfinex'])
while len(list_of_coin_exchange_combination_not_updated) != 0:
    list_of_coin_exchange_combination_not_updated2 = []
    i,j=0,0
    for not_updated in list_of_coin_exchange_combination_not_updated:
        list_of_coin_exchange_combination_not_updated2,lst_new_data,j,i = download_incremental_data(list_of_coin_exchange_combination_not_updated2,lst_new_data,j,i,
                                coin=not_updated['coin'],
                              exchange=not_updated['exchange'],
                        nperiods_ago=not_updated['nperiods'],
                        period=not_updated['period']
                        )
    list_of_coin_exchange_combination_not_updated = list_of_coin_exchange_combination_not_updated2
append_to_csv(lst_new_data,csv_column_order,final_csv_column_order,csv_filename,datetimeStringformat_to_csv,df_coin_period)

2018-08-11 12:44:17,248 - INFO - Exchange        Last Updated Time            Elapsed Time nPeriodsAgo
2018-08-11 12:44:17,266 - INFO - ---------- For coin - BTC
2018-08-11 12:44:17,268 - INFO - Bitfinex        1533700800                         270857       75.0
2018-08-11 12:44:17,270 - INFO - Updating data for BTC-Bitfinex from 1533700800


Updated Dataframe
Done


In [ ]:
indicator_list_btc = ['unix_timestamp','UPPER_BOLLINGER_BAND_VALUE','MIDDLE_BOLLINGER_BAND_VALUE','LOWER_BOLLINGER_BAND_VALUE']

In [ ]:
def update_indicator_BTC(csv_filename,periods,timeframe,datetimeformat_string):
    """ Update the given csv_file with new column values for corr rows """
    df_csv = pd.read_csv(csv_filename, index_col=None,dayfirst=True)
    #df_csv.time = df_csv.time.apply(lambda t: datetime.datetime.strptime(t, '%d-%m-%Y %H:%M:%S'))
    
    for indicator in indicator_list_btc:
        if indicator not in df_csv.columns and indicator not in df_csv.index:
            df_csv[indicator] = np.nan
    #df_csv.unix_timestamp =  df_csv.time.apply(lambda t: time.mktime(datetime.datetime.strptime(str(t) ,'%d-%m-%Y %H:%M').timetuple()))
    df_csv = df_csv.set_index(['coin', 'exchange','unix_timestamp'])
    data = list(df_csv.index.get_level_values(0).unique())
    i=0
    j=0
    for coin_name in data:
        coin_df = df_csv[df_csv.index.get_level_values(0)==coin_name] 
        #print(coin_df)
        coin_df = coin_df.reset_index()
        coin_df = coin_df.sort_values(by=['exchange','unix_timestamp']).set_index(['coin', 'exchange','unix_timestamp'])
        #print(coin_df)
        df_groupby = coin_df.groupby(['exchange'], group_keys=False)
        for key, item in df_groupby:
            req_data = df_groupby.get_group(key)
            req_data2 = req_data.iloc[-periods:]

            start_date = req_data2.index.get_level_values(2)[0]
            end_date = req_data2.index.get_level_values(2)[req_data2.shape[0]-1]
            req_data2 = req_data[(req_data.index.get_level_values(2) >= start_date) & (req_data.index.get_level_values(2) <= end_date)]
            #print(req_data2)
            np_volumeto = np.array(req_data2.volumeto.values,dtype='f8')
            if len(np_volumeto)<20:
                j = j+1
                print(coin_name,j," Not Updated")
                continue
            req_data2['UPPER_BOLLINGER_BAND_VALUE'] = pyti.bollinger_bands.upper_bollinger_band(req_data2.close.values,20)
            req_data2['MIDDLE_BOLLINGER_BAND_VALUE'] = pyti.bollinger_bands.middle_bollinger_band(req_data2.close.values,20)
            req_data2['LOWER_BOLLINGER_BAND_VALUE'] = pyti.bollinger_bands.lower_bollinger_band(req_data2.close.values,20)
            df_csv.update(req_data2)
            i = i+1
            print(coin_name,i)
            #print(df_csv.query('coin == @coin_name').tail(1))
            #sys.exit("Testing")
    df_csv.to_csv(csv_filename,date_format=datetimeformat_string)
    print("Done")

In [ ]:
update_indicator_BTC(data_directory+'BTC_Bitfinex_day_full_1day.csv',250,'1day',datetimeStringformat_to_csv)
print("Done")  

In [ ]:
update_indicator_BTC(data_directory+'BTC_Bitfinex_day_full_3days.csv',250,'3day',datetimeStringformat_to_csv)
print("Done")  

In [ ]:
update_indicator_BTC(data_directory+'BTC_Bitfinex_day_full_7days.csv',250,'7day',datetimeStringformat_to_csv)
print("Done")

In [ ]:
update_indicator_BTC(data_directory+'BTC_Bitfinex_day_full_14days.csv',250,'14day',datetimeStringformat_to_csv)
print("Done")

In [ ]:
update_indicator_BTC(data_directory+'BTC_Bitfinex_hour_full_1hour.csv',250,'1hour',datetimeStringformat_to_csv)
print("Done")

In [ ]:
update_indicator_BTC(data_directory+'BTC_Bitfinex_hour_full_4hours.csv',250,'4hour',datetimeStringformat_to_csv)
print("Done")

In [ ]:
update_indicator_BTC(data_directory+'BTC_Bitfinex_hour_full_6hours.csv',250,'6hour',datetimeStringformat_to_csv)
print("Done")

In [ ]:
update_indicator_BTC(data_directory+'BTC_Bitfinex_hour_full_12hours.csv',250,'12hour',datetimeStringformat_to_csv)
print("Done")

In [137]:
indicator_list_histogram = ['Count','Division']

In [138]:
def create_Histogram(csv_filename,periods,timeframe,datetimeformat_string,output_file_name,number_of_bins_to_create):
    """ Update the given csv_file with new column values for corr rows """
    df_csv = pd.read_csv(csv_filename, index_col=None,dayfirst=True)
    df_csv = df_csv.set_index(['coin', 'exchange','unix_timestamp'])
    data = list(df_csv.index.get_level_values(0).unique())
    i=0
    j=0
    lst = []
    for coin_name in data:
        coin_df = df_csv[df_csv.index.get_level_values(0)==coin_name] 
        #print(coin_df)
        coin_df = coin_df.reset_index()
        coin_df = coin_df.sort_values(by=['exchange','unix_timestamp']).set_index(['coin', 'exchange','unix_timestamp'])
        #print(coin_df)
        df_groupby = coin_df.groupby(['exchange'], group_keys=False)
        for key, item in df_groupby:
            req_data = df_groupby.get_group(key)
            df = {}
            df['coin'] = coin_name
            df['exchange'] = key
            list_of_tuples = create_List_of_Tuples(req_data.close,number_of_bins_to_create)
            df['Histogram_Close'] = list_of_tuples
            list_of_tuples =  create_List_of_Tuples(req_data.high,number_of_bins_to_create)
            df['Histogram_High'] = list_of_tuples
            list_of_tuples =  create_List_of_Tuples(req_data.low,number_of_bins_to_create)
            df['Histogram_Low'] = list_of_tuples
            dataframe = pd.DataFrame(df)
            lst.append(dataframe)
    df_new_data = pd.concat(lst)
    df_new_data.to_csv(output_file_name)

In [139]:
def create_List_of_Tuples(price_series,number_of_bins):
    count,division = np.histogram(price_series,bins=number_of_bins)
    #print(type(count),type(division))
    #print(count.tolist(),division.tolist())
    #str1 = np.array_str(count)
    #str2 = np.array_str(division)
    division_list = division.tolist()
    count_list = count.tolist()
    list_of_tuples = []
    for idx,value in enumerate(division_list):
        if idx == len(division_list)-1:
            break
        range_tuple = (division_list[idx],division_list[idx+1],count_list[idx])
        list_of_tuples.append(range_tuple)
    return list_of_tuples

In [140]:
create_Histogram(data_directory+'BTC_Bitfinex_hour_full_1hour.csv',250,'1hour',datetimeStringformat_to_csv,data_directory+'BTC_Bitfinex_hour_full_1hourHistogram.csv',10)
print("Done")

Done


In [141]:
create_Histogram(data_directory+'BTC_Bitfinex_hour_full_4hours.csv',250,'4hour',datetimeStringformat_to_csv,data_directory+'BTC_Bitfinex_hour_full_4hoursHistogram.csv',10)
print("Done")

Done


In [142]:
create_Histogram(data_directory+'BTC_Bitfinex_hour_full_6hours.csv',250,'6hour',datetimeStringformat_to_csv,data_directory+'BTC_Bitfinex_hour_full_6hoursHistogram.csv',10)
print("Done")

Done


In [143]:
create_Histogram(data_directory+'BTC_Bitfinex_hour_full_12hours.csv',250,'12hour',datetimeStringformat_to_csv,data_directory+'BTC_Bitfinex_hour_full_12hoursHistogram.csv',10)
print("Done")

Done


In [144]:
create_Histogram(data_directory+'BTC_Bitfinex_day_full_1day.csv',250,'1day',datetimeStringformat_to_csv,data_directory+'BTC_Bitfinex_day_full_1dayHistogram.csv',10)
print("Done")

Done


In [145]:
create_Histogram(data_directory+'BTC_Bitfinex_day_full_3days.csv',250,'3days',datetimeStringformat_to_csv,data_directory+'BTC_Bitfinex_day_full_3daysHistogram.csv',10)
print("Done")

Done


In [146]:
create_Histogram(data_directory+'BTC_Bitfinex_day_full_7days.csv',250,'7days',datetimeStringformat_to_csv,data_directory+'BTC_Bitfinex_day_full_7daysHistogram.csv',15)
print("Done")

Done


In [147]:
create_Histogram(data_directory+'BTC_Bitfinex_day_full_14days.csv',250,'14days',datetimeStringformat_to_csv,data_directory+'BTC_Bitfinex_day_full_14daysHistogram.csv',10)
print("Done")

Done


In [ ]:
create_Histogram(data_directory+'all_coins_day_full_1day.csv',250,'1day',datetimeStringformat_to_csv,data_directory+'all_coins_day_full_1dayHistogram.csv')
print("Done")

In [ ]:
scheduler = BackgroundScheduler()
scheduler.start()
columns_order = ['coin','exchange','unix_timestamp','time','open','high','low','close','volumefrom','volumeto',
                 'UPPER_BOLLINGER_BAND_VALUE','MIDDLE_BOLLINGER_BAND_VALUE','LOWER_BOLLINGER_BAND_VALUE']
scheduler.add_job(update_csv_to_latest, "cron", ['1daybtcbitfinex',datetimeStringformat_to_csv,columns_order], minute='*/1')


In [ ]:
scheduler.shutdown(wait=False)

In [ ]:
def changeCSVDateTimeFormat(csv_filename):
    df_csv = pd.read_csv(csv_filename, index_col=None,dayfirst=True,infer_datetime_format =True)
    #df_csv.unix_timestamp =  df_csv.time.apply(lambda t: time.mktime(datetime.datetime.strptime(str(t) ,'%d-%m-%Y %H:%M').timetuple()))
    df_csv['time'] = pd.to_datetime(df_csv.unix_timestamp,unit="s",utc=True)
    df_csv.set_index(['coin', 'exchange','unix_timestamp']).to_csv(csv_filename,date_format=datetimeStringformat_to_csv)
    print("Done")

In [ ]:
changeCSVDateTimeFormat('all_coins_day_full_1day.csv')

In [ ]:
changeCSVDateTimeFormat('all_coins_day_full_1day_Cryptopia.csv')

In [ ]:
changeCSVDateTimeFormat('BTC_Bitfinex_day_full_1day.csv')

In [ ]:
changeCSVDateTimeFormat('BTC_Bitfinex_hour_full_1hour.csv')

In [ ]:
print( type(datetime.datetime.now()))

In [ ]:
df_csv = pd.read_csv(data_directory+'all_coins_day_full_1day.csv', index_col=None,dayfirst=True)
df_csv = df_csv.set_index(['coin', 'exchange','unix_timestamp'])
data = list(df_csv.index.get_level_values(0).unique())
i=0
j=0
lst = []
coin_df = df_csv[df_csv.index.get_level_values(0)=='VEN']
coin_df.open = coin_df.open/100
coin_df.close = coin_df.close/100
coin_df.high = coin_df.high/100
coin_df.low = coin_df.low/100
df_csv.update(coin_df)
df_csv.to_csv(data_directory+'all_coins_day_full_1day.csv')